<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *DATA SCIENCE / SECTION 4 / SPRINT 2 / NOTE 2*

# 📝 Assignment

---


# Distributed_Representation

## 1. Multiple Choice


### 문항 1) 다음 보기 중 <ins>옳은</ins> 것을 모두 고르시오.

1. 분포 가설(Distributional hypothesis)이란 '비슷한 위치에서 등장하는 단어들은 비슷한 의미를 가진다'는 의미의 가설입니다.
2. 원-핫 인코딩은 가장 간단한 방법인데다 단어 간 유사도도 구하기 쉽기 때문에 단어를 벡터화하는 데 자주 사용됩니다.
3. (같은 모델을 사용하여) 임베딩으로 표현한 벡터는 모두 동일한 차원을 가집니다. 
4. 임베딩 벡터의 원소는 연속적인 값을 가진다는 특징을 가지고 있습니다.
5. 정답이 없다.

### 문항 2) Word2Vec에 대한 설명 중 <ins>옳지 않은</ins> 것을 모두 고르시오.

1. CBoW는 주변 단어의 정보를 바탕으로 중심 단어를 맞추는 방식으로 학습하고,<br/>
Skip-gram은 중심 단어의 정보를 바탕으로 주변 단어를 맞추는 방식으로 학습합니다.
2. Word2Vec의 기본 구조는 은닉층이 1개인 신경망입니다.
3. 이 때, 은닉층의 노드 개수는 출력 벡터의 차원을 결정합니다. 
4. Word2Vec으로 생성한 임베딩 벡터는 단어의 의미적, 문법적(구조적) 관계를 잘 보존하고 있습니다.
5. 정답이 없다.

### 문항 3) fastText에 대한 설명 중 <ins>옳지 않은</ins> 것을 모두 고르시오.

1. OOV(Out of Vocabulary) 문제란 말뭉치에 등장하지 않은 단어를 생성할 때 에러가 발생하는 현상입니다.
2. fastText는 철자 단위(Character-level) 임베딩을 사용하여 OOV 문제를 해결하였습니다.
3. fastText는 철자 단위 임베딩 덕분에 접두사, 접미사 등의 관계도 잘 알아내는 특징을 가지고 있습니다.
4. fastText의 결과로 나온 단어 임베딩 벡터는 단어의 문법적(구조적) 관계에 조금 더 집중하는 경향이 있습니다. 
5. 정답이 없다.

## 2. Code

캐글의 [SMS Spam dataset](https://www.kaggle.com/uciml/sms-spam-collection-dataset) 에 사전 학습된 Word2Vec 임베딩 벡터를 적용하여 분류해봅시다.<br/>
세션 노트에 있었던 단어 임베딩 벡터를 평균내어 분류하는 방법을 적용해봅시다.

In [ ]:
!pip install gensim --upgrade

Requirement already up-to-date: gensim in /usr/local/lib/python3.7/dist-packages (4.0.1)


In [ ]:
import gensim

gensim.__version__

'4.0.1'

In [ ]:
import gensim.downloader as api

wv = api.load('word2vec-google-news-300')

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer

### 1) 데이터 전처리
    
- 데이터셋을 데이터프레임으로 읽어옵니다 `encoding = 'latin-1'` 을 사용합니다.
- 필요없는 열(column)을 삭제합니다.
- LabelEncoder를 사용하여 label 전처리를 해줍니다. 

In [ ]:
from google.colab import files

file = files.upload()

Saving spam.csv to spam.csv


In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
### 이곳에서 과제를 수행해 주세요 ###

### 2) 텍스트 분류를 수행해주세요.

- 데이터셋 split시 test_size의 비율은 15%로, `random_state = 42` 로 설정합니다. 
- Tokenizer의 `num_words = 1000` 으로 설정합니다.
- pad_sequence의 `maxlen=150` 으로 설정합니다.
- 학습 시, 파라미터는 `batch_size=64, epochs=10, validation_split=0.2` 로 설정합니다.
- evaluate 했을 때의 loss와 accuarcy를 [loss, acc] 형태로 입력해주세요. Ex) [0.4321, 0.8765]

In [ ]:
### 이곳에서 과제를 수행해 주세요 ###

## 3. 도전과제 🔥


### Count-based Representation을 활용한 방법과 비교해봅시다.

- 지난 시간에 배운 `CountVectorizer`, 혹은 `TfidfVectorizer` 를 활용하여<br/>
위에서 했던 데이터셋에 분류 태스크를 수행해봅시다.

- [Link](https://www.kaggle.com/faressayah/natural-language-processing-nlp-for-beginners) 에 있는 노트북을 참고하여 `Multinomial NB` 를 적용해보세요.

- Count-based Representation 방법과 Distributed Representation 방법 중<br/>
어떤 방법의 성능이 더 좋은가요?

In [ ]:
### 이곳에서 과제를 수행해 주세요 ###